In [1]:
# Creating LLM Application using LCEL
# LCEL (Language Chain Expression Language) is a high-level language designed to simplify the creation of applications that leverage large language models (LLMs).
# First we need to create a key like that done foe OpenAI and add to the .env 

import os
from dotenv import load_dotenv
load_dotenv()

import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

groq_api_key = os.getenv("GROQ_API_KEY")

In [10]:
from langchain_groq import ChatGroq                    
model = ChatGroq(model= "llama-3.1-8b-instant", groq_api_key=groq_api_key)  # specify the model name
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000001BC49070D90>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001BC49BE5290>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [12]:
from langchain_core.messages import HumanMessage, SystemMessage         # need to provide different types of messages to the llm model
messages = [
    SystemMessage(content="You are a helpful assistant that translates English to Spanish."), # system message to set the behavior of the model
    HumanMessage(content="Translate the following English text to Spanish: 'Hello, how are you?'") # human message with the input text
]

result = model.invoke(messages)  # invoke the model with the messages


In [14]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
parser.invoke(result)  # parse the result to get the final output

"La traducción del texto inglés 'Hello, how are you?' al español es:\n\n'Hola, ¿cómo estás?'"

In [15]:
# Using LCEL - chain the components together in a single expression
chain = model | parser
chain.invoke(messages)  # directly invoke the chain with the messages

'La traducción al español es: "Hola, ¿cómo estás?"'

In [17]:
# Prompt Templates - efficient way to manage prompts
from langchain_core.prompts import ChatPromptTemplate
generic_template = "Translate the following English text to {language}: "

prompt = ChatPromptTemplate.from_messages(
    [("system", generic_template), ("user", "{text}")]
)

In [20]:
result = prompt.invoke({"language": "Spanish", "text": "Hello, how are you?"})   # converting the template into a list of messages like done above

In [21]:
result.to_messages()

[SystemMessage(content='Translate the following English text to Spanish: ', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={})]

In [26]:
# Chaining all together using LCEL 
# Prompt -> Model -> Parser
chain = prompt | model | parser
chain.invoke({"language": "Spanish", "text": "Hello, how are you?"})

'Hola, ¿cómo estás?'